In [1]:
from keras import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import pickle as pkl
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
DATA_DIR = 'signatures_merging/'

In [4]:
cv_data_dir = 'ddi_split_perc_drugs/'

In [5]:
h_df = pkl.load(open(DATA_DIR+'h_df.p','rb'))

In [ ]:
h_df.head()

In [ ]:
s_df = pkl.load(open(DATA_DIR+'s_df.p','rb'))

In [ ]:
s_df.head()

In [39]:
merged_df = pkl.load(open(DATA_DIR+'merged_df_full.p','rb'))

In [40]:
merged_df.head()

,stitch_id1,stitch_id2,drug1,drug2,event_umls_id,event_name,drug_x,s1_x,s2_x,s3_x,...,s5,s6,s7,s8,s9,s10,umls_id,cid1.cid2-name1.name2,umls_id-se,val
0,191,772,adenosine,heparin,C0003578,apnea,adenosine,0.00000,25.30435,0.00000,...,0.04236,0.0,0.0,0.0,0.0,0.01458,C0003578,191-772-adenosine-heparin,C0003578-apnea-apnoea,1
1,444,1983,bupropion,acetaminophen,C0003578,apnea,bupropion,9.61308,5.48765,18.27944,...,0.04236,0.0,0.0,0.0,0.0,0.01458,C0003578,444-1983-bupropion-paracetamol,C0003578-apnea-apnoea,1
2,1302,1983,naproxen,acetaminophen,C0003578,apnea,naproxen,0.00000,0.00000,0.00000,...,0.04236,0.0,0.0,0.0,0.0,0.01458,C0003578,1302-1983-naproxen-paracetamol,C0003578-apnea-apnoea,1
3,191,2083,adenosine,salbutamol,C0003578,apnea,adenosine,0.00000,25.30435,0.00000,...,0.04236,0.0,0.0,0.0,0.0,0.01458,C0003578,191-2083-adenosine-salbutamol,C0003578-apnea-apnoea,1
4,444,2083,bupropion,salbutamol,C0003578,apnea,bupropion,9.61308,5.48765,18.27944,...,0.04236,0.0,0.0,0.0,0.0,0.01458,C0003578,444-2083-bupropion-salbutamol,C0003578-apnea-apnoea,1


In [41]:
x_df = merged_df.pivot_table(index='cid1.cid2-name1.name2',columns='umls_id-se',values='val')

In [42]:
x_df.fillna(0.0,inplace=True)

In [43]:
top_se_y = x_df[pd.DataFrame(x_df.sum()).idxmax()].values


In [44]:
top_se_y.shape

(36143, 1)

In [45]:
top_se_y

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [0.],
       [1.]])

In [ ]:
Y = x_df.values

In [ ]:
Y.shape

In [ ]:
Y[:,sorted(Y.sum(axis=0).argsort()[-1:][::1])]

In [ ]:
Y[:,207].

In [ ]:
for i in range(Y.shape[1]):
    if Y[:,i].sum() == 16407.0:
        print(i)
        break

In [ ]:
h_df.head()

In [ ]:
S = s_df.values
S.shape

In [6]:
H = h_df.values
H.shape

(544, 10)

In [7]:
H.T.shape

(10, 544)

In [8]:
h_weights = [H.T]

In [ ]:
len(h_weights)

In [ ]:
h_weights[0][:,207].shape

In [ ]:
normed_h = h_weights/np.amax(h_weights)

In [ ]:
normed_s = S/np.amax(S)

In [9]:
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from keras import regularizers

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(S, Y, test_size=0.1, shuffle= False)

In [ ]:
print(x_train.shape)
print(x_valid.shape)
print(y_train.shape)
print(y_valid.shape)

### GETTING THE SPLIT LIKE ZHANG AND JIN

In [10]:
x_train = pkl.load(open(cv_data_dir+'training_set_features_S.p','rb'))
y_train = pkl.load(open(cv_data_dir+'training_set_labels_X.p','rb'))
x_test = pkl.load(open(cv_data_dir+'testing_set_features_S.p','rb'))
y_test = pkl.load(open(cv_data_dir+'testing_set_labels_X.p','rb'))

### Model initialisation

In [11]:
from keras.layers import Activation, Dropout
from keras.initializers import RandomNormal
from keras.utils.generic_utils import get_custom_objects
from keras.callbacks import EarlyStopping
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [12]:
from sklearn.metrics import roc_auc_score, average_precision_score

In [13]:
def custom_activation(x):
    return x
get_custom_objects().update({'custom_activation': Activation(custom_activation)})

In [ ]:
model = Sequential()
model.add(Dense(35, activation='sigmoid', kernel_initializer='random_normal', kernel_regularizer=regularizers.l2(10), input_shape=(20,)))
# model.add(Dropout(0.2))
model.add(Dense(20, activation='relu', kernel_initializer='random_normal'))
# model.add(Dropout(0.5))
model.add(Dense(10, activation='sigmoid', kernel_initializer='random_normal', kernel_regularizer=regularizers.l2(1)))
# model.add(Dense(10, activation='relu', kernel_initializer='random_normal'))
#Output Layer
# model.add(Dense(544, activation='sigmoid',kernel_initializer='random_normal', kernel_regularizer=regularizers.l2(10)))
fixed_layer = Dense(544,activation=custom_activation,use_bias=False)
fixed_layer.trainable = False
model.add(fixed_layer)
model.layers[-1].set_weights(h_weights)

In [ ]:
model.summary()

In [23]:
epochs = 50
batch_size = 128

## CROSS VALIDATION

In [ ]:
# this form of splitting doesn't account for the testing setting
folds = list(KFold(n_splits=5, shuffle=True, random_state=1).split(x_train, y_train))

In [15]:
cv_splits = pkl.load(open(cv_data_dir+'cv_splits.p','rb'))

In [ ]:
auc_scores = []
aupr_scores = []
for j, (train_idx, val_idx) in enumerate(folds):
    print('\nFold ',j)
    x_train_cv = x_train[train_idx]
    y_train_cv = y_train[train_idx]
    x_valid_cv = x_train[val_idx]
    y_valid_cv= y_train[val_idx]
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    history = model.fit(x_train_cv,y_train_cv,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1)
    predicted = model.predict(x_valid_cv)
    auc_scores.append(roc_auc_score(y_valid_cv.flatten(),predicted.flatten()))
    aupr_scores.append(average_precision_score(y_valid_cv.flatten(),predicted.flatten()))

In [ ]:
sum(auc_scores)/len(auc_scores)

In [ ]:
sum(aupr_scores)/len(aupr_scores)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
history = model.fit(x_train,y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1)

### Prepare for grid search best hypParams

In [16]:
def create_model(neurons_lay2=50, neurons_extra_layer = 20, l2_reg=0.0, activ_l2='tanh', extra_layers=2):
    # Instantiate model
    model = Sequential()
    # Layer 2
    model.add(Dense(neurons_lay2, activation=activ_l2, kernel_initializer='random_normal', kernel_regularizer=regularizers.l2(l2_reg), input_shape=(20,)))

    for i in range(extra_layers):
        model.add(Dense(neurons_extra_layer, activation='relu', kernel_initializer='random_normal', kernel_regularizer=regularizers.l2(l2_reg)))
    
    model.add(Dense(10, activation='sigmoid', kernel_initializer='random_normal', kernel_regularizer=regularizers.l2(l2_reg)))
    # Fixed output layer
    fixed_layer = Dense(544,activation=custom_activation,use_bias=False)
    fixed_layer.trainable = False
    model.add(fixed_layer)
    model.layers[-1].set_weights(h_weights)
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model


In [ ]:
early_stopper=EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')

In [ ]:
model = KerasClassifier(build_fn=create_model, nb_epoch=10, batch_size=99, verbose=0)

In [ ]:
(50, 90, 0.0, 'sigmoid', 'relu', 'sigmoid', True)

In [17]:
# # Which parameters we choose here is more a gut decision.

neurons_lay2=[20,50,100]
neurons_extra_layer=[50,100]
l2_reg=[0.0]
activ_l2=['sigmoid','tanh']
extra_layers=[0,2,5]

In [20]:
import itertools

In [21]:
grid_params = list(itertools.product(neurons_lay2,neurons_extra_layer,l2_reg,activ_l2,extra_layers))
auc_scores = []
aupr_scores = []

In [22]:
len(grid_params)

36

In [ ]:
neurl2 = 0
neur_extr = 1
l2 = 2
actL2 = 3
extra = 4
auc_scores_grid = []
aupr_scores_grid = []
for i, param_comb in enumerate(grid_params):
    auc_scores = []
    aupr_scores = []
    model = create_model(param_comb[neurl2],param_comb[neur_extr],param_comb[l2], param_comb[actL2],param_comb[extra])
    for j, fold in enumerate(cv_splits):
        print('Fold {}\n'.format(j+1))
        x_train_cv = fold['training_set_features_S']
        y_train_cv = fold['training_set_labels_X']
        x_valid_cv = fold['test_set_features_S']
        y_valid_cv= fold['test_set_labels_X']
        
        history = model.fit(x_train_cv,y_train_cv,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=0)
        predicted = model.predict(x_valid_cv)
        auc_scores.append(roc_auc_score(y_valid_cv.flatten(),predicted.flatten()))
        aupr_scores.append(average_precision_score(y_valid_cv.flatten(),predicted.flatten()))

    auc_scores_grid.append(sum(auc_scores)/len(auc_scores))
    aupr_scores_grid.append(sum(aupr_scores)/len(aupr_scores))
    print('Percentage done: {}% - Iterations done: {}/{}'.format(((i+1)/len(grid_params)*100),i+1,len(grid_params)))

In [ ]:
max(aupr_scores_grid)

In [ ]:
max(auc_scores_grid)

In [ ]:
grid_params[auc_scores_grid.index(max(auc_scores_grid))]

In [ ]:
grid_params[aupr_scores_grid.index(max(aupr_scores_grid))]

In [ ]:
auc_scores_grid

In [25]:
model = create_model(100, 100, 0.0, 'tanh', 5)
# model = create_model(120, 90, 0.0, 'tanh', 'relu', 'sigmoid', True)
history = model.fit(x_train,y_train,
                batch_size=batch_size,
                epochs=epochs,
                verbose=1)
predicted = model.predict(x_test)
print(roc_auc_score(y_test.flatten(),predicted.flatten()))
print(average_precision_score(y_test.flatten(),predicted.flatten()))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
29103/29103 [==============================] - 3s 95us/step - loss: 0.3138 - acc: 0.8933
Epoch 2/50
29103/29103 [==============================] - 2s 62us/step - loss: 0.3098 - acc: 0.8933
Epoch 3/50
29103/29103 [==============================] - 2s 65us/step - loss: 0.3068 - acc: 0.8933
Epoch 4/50
29103/29103 [==============================] - 2s 60us/step - loss: 0.3045 - acc: 0.8933
Epoch 5/50
29103/29103 [==============================] - 2s 59us/step - loss: 0.3026 - acc: 0.8933
Epoch 6/50
29103/29103 [==============================] - 2s 55us/step - loss: 0.3017 - acc: 0.8933
Epoch 7/50
29103/29103 [==============================] - 2s 58us/step - loss: 0.3004 - acc: 0.8934
Epoch 8/50
29103/29103 [==============================] - 2s 57us/step - loss: 0.2996 - acc: 0.8934
Epoch 9/50
29103/29103 [==============================] - 2s 55us/step - loss: 0

# --- Jin's testing on random columns and top 50 SEs ---

In [29]:
y_train.shape

(29103, 544)

In [36]:
pd.DataFrame(y_train).sum()

0       6050.0
1      10205.0
2       3387.0
3       3945.0
4       3072.0
        ...   
539      307.0
540     2036.0
541     2801.0
542      169.0
543     3261.0
Length: 544, dtype: float64

In [37]:
def nlargest_cols(a, n):
    return a[:, sorted(a.sum(axis=0).argsort()[-n:][::-1])]

In [38]:
nlargest_cols(y_train,50)

array([[0., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [60]:
top50_se_idx = x_df.sum().sort_values(ascending=False).index[:50]

In [55]:
y_train_df = pd.DataFrame(y_train)

In [57]:
y_train_df.columns = x_df.columns

In [61]:
y_train_df[top50_se_idx]

umls_id-se,C0020649-arterial pressure nos decreased-hypotension,C0002871-anaemia-anaemia,C0013404-difficulty breathing-dyspnoea,C0027497-nausea-nausea,C0015672-fatigue-fatigue,C0032285-neumonia-pneumonia,C0030193-pain-pain,C0011991-diarrhea-diarrhoea,C0004093-asthenia-asthenia,C0042963-emesis-vomiting,...,C0004238-afib-atrial fibrillation,C0038454-apoplexy-cerebrovascular accident,C0015230-eruption-rash,C0917801-insomnia-insomnia,C0027947-neutropenia-neutropenia,C0002622-amnesia-amnesia,C0011168-deglutition disorder-dysphagia,C0017168-acid reflux-gastrooesophageal reflux disease,C0040822-tremor-tremor,C0231218-feeling unwell-malaise
0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
3,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
6,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
8,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0


In [65]:
predicted = model.predict(x_test)
y_test_df = pd.DataFrame(y_test)

In [67]:
predicted_df = pd.DataFrame(predicted)

In [68]:
predicted_df.columns = x_df.columns
y_test_df.columns = x_df.columns

In [146]:
print(roc_auc_score(y_test_df[top50_se_idx].values.flatten(),predicted_df[top50_se_idx].values.flatten()))
print(average_precision_score(y_test_df[top50_se_idx].values.flatten(),predicted_df[top50_se_idx].values.flatten()))

0.5908932066541042
0.402867368173999


In [81]:
test_50 = y_test_df[top50_se_idx].values
predicted_50 = predicted_df[top50_se_idx].values
aurocs = []
auprs = []
for i in range(y_test.shape[1]):
    aurocs.append(roc_auc_score(y_test[:,i],predicted[:,i]))
    auprs.append(average_precision_score(y_test[:,i],predicted[:,i]))


In [88]:
import random

In [132]:
sum(random.sample(aurocs,2))/2

0.5674143727848062

In [139]:
random_aurocs = [sum(x)/2 for x in itertools.combinations(aurocs,2)]

In [145]:
max(random_aurocs)

0.8367502842524162

In [142]:
random_auprs = [sum(x)/2 for x in itertools.combinations(auprs,2)]

In [144]:
max(random_auprs)

0.5274052708578866

In [74]:
test_50[:,1]

array([1., 1., 0., ..., 0., 0., 1.])

### ------------------------------------------

In [ ]:
max(auc_scores)

In [ ]:
score = model.evaluate(x_valid, y_valid, verbose=0)
# score
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
predicted = model.predict(x_valid)

In [ ]:
hist = pd.DataFrame(model.layers[1].get_weights()[0].flatten()).hist()

In [ ]:
x_df.sum(axis=1)

In [ ]:
sns.distplot(sorted(list(x_df.sum(axis=1))))

In [ ]:
x_df.sum(axis=1).hist()

In [ ]:
roc_auc_score(y_valid.flatten(),predicted.flatten())

In [ ]:
average_precision_score(y_valid.flatten(),predicted.flatten())

### Decagon's testing procedure

In [ ]:
def create_model_single_se(neurons_lay2=25, neurons_extra_layer = 20, l2_reg=0.0, activ_l2='tanh', activ_l3='sigmoid', activ_l4='sigmoid', extra_layer=False,h_weights_col = 0):
    # Instantiate model
    model = Sequential()
    # Layer 2
    model.add(Dense(neurons_lay2, activation=activ_l2, kernel_initializer='random_normal', kernel_regularizer=regularizers.l2(l2_reg), input_shape=(20,)))
    # if extra layer == True
    if extra_layer:
        model.add(Dense(neurons_extra_layer, activation=activ_l3, kernel_initializer='random_normal', kernel_regularizer=regularizers.l2(l2_reg)))
        model.add(Dense(10, activation=activ_l4, kernel_initializer='random_normal', kernel_regularizer=regularizers.l2(l2_reg)))
    else:
        # Layer 3 or 4 if else
        model.add(Dense(10, activation=activ_l3, kernel_initializer='random_normal', kernel_regularizer=regularizers.l2(l2_reg)))
    # Fixed output layer
    model.add(Dense(1, activation='sigmoid',kernel_initializer='random_normal'))
#     fixed_layer = Dense(1,activation=custom_activation,use_bias=False)
#     fixed_layer.trainable = False
#     model.add(fixed_layer)
#     model.layers[-1].set_weights([h_weights[0][:,h_weights_col].reshape(10,1)])
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model


In [ ]:
h_weights[0][:,207].reshape(10,1)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(S, top_se_y, stratify=top_se_y, test_size=0.1)

In [ ]:
x_tr_proper, x_valid, y_tr_proper, y_valid = train_test_split(x_train,y_train, stratify=y_train, test_size=0.1)

In [ ]:
# for i in range(y_valid.shape[1]):

model = create_model_single_se(5, 20, 0.0, 'sigmoid', 'relu', 'relu', False, 207)
history = model.fit(x_tr_proper,y_tr_proper,
                batch_size=batch_size,
                epochs=epochs,
                verbose=1)
predicted = model.predict(x_valid)
print(roc_auc_score(y_valid.flatten(),predicted.flatten()))
print(average_precision_score(y_valid.flatten(),predicted.flatten()))

In [ ]:
predicted = model.predict(x_test)
print(roc_auc_score(y_test.flatten(),predicted.flatten()))
print(average_precision_score(y_test.flatten(),predicted.flatten()))

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

### -------

In [ ]:
pred = pd.DataFrame(predicted.flatten())

In [ ]:
true = pd.DataFrame(y_valid.flatten())

In [ ]:
true[true[0]>0].index

In [ ]:
pred[pred.index.isin(true[true[0]>0].index)].hist()

In [ ]:
pred[pred.index.isin(true[true[0]==0].index)].hist()

In [ ]:
ones = np.array(pred[pred.index.isin(true[true[0]>0].index)])
zeros = np.array(pred[pred.index.isin(true[true[0]==0].index)])

In [ ]:
plt.hist(ones, alpha=0.5, label='ones')
plt.hist(zeros, alpha=0.5, label='zeros')
plt.legend(loc='upper right')
plt.show()

In [ ]:
sns.distplot(ones, hist = True, kde = True,
                 kde_kws = {'linewidth': 3},
                 label = 'ones')
sns.distplot(zeros, hist = True, kde = True,
                 kde_kws = {'linewidth': 3},
                 label = 'zeros')
plt.legend(prop={'size': 16}, title = 'pdf-ddi')
# plt.title('Density Plot with Multiple Airlines')
plt.xlabel('pred_values')
plt.ylabel('Density')
plt.savefig('density-ddi.png')